In [1]:
import pandas as pd
import numpy as np
import csv
import os
import time


# Generate the V or X Matrix from NYT COVID time-series

In [2]:
"""
Extract the time series of the total number of confirmed cases/deaths in each state in the original data set.

parameters settings

days: the number of days collected

"""

'\nExtract the time series of the total number of confirmed cases/deaths in each state in the original data set.\n\nparameters settings\n\ndays: the number of days collected\n\n'

In [3]:
days = 623 # Mar 18,2020 - Nov 30,2021

In [4]:
## confirmed covid-19 cumulative cases/deaths
input_original_data = 'us-states.csv'

#output_confirmed_inter = 'covid19_NYT_confirmed_US_51_states_inter.csv'
output_confirmed_inter = 'covid19_NYT_deaths_US_51_states_inter.csv'

#output_confirmed_cases_US = 'covid19_NYT_confirmed_US_51_states_X_matrix_final.csv'
output_confirmed_cases_US = 'covid19_NYT_deaths_US_51_states_X_matrix_final.csv'


In [5]:
pd.set_option("max_columns", None)
pd.set_option("max_colwidth", None)

df1 = pd.read_csv(input_original_data)

df1.head(100)


,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
...,...,...,...,...,...
95,2020-02-12,California,6,7,0
96,2020-02-12,Illinois,17,2,0
97,2020-02-12,Massachusetts,25,1,0
98,2020-02-12,Texas,48,1,0


In [6]:
# We included Washington D.C. as a node for the graph similar to the states of US
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
          'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
          'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
          'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
          'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
          'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
          'New Jersey', 'New Mexico', 'New York', 'North Carolina',
          'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
          'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
          'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia',
          'Wisconsin', 'Wyoming']

len(states)

51

In [7]:
df2 = (df1[(df1['date'] >= '2020-03-17') & (df1['date'] <= '2021-11-30')]).reset_index(drop=True)
df2.head(100)

,date,state,fips,cases,deaths
0,2020-03-17,Alabama,1,39,0
1,2020-03-17,Alaska,2,6,0
2,2020-03-17,Arizona,4,20,0
3,2020-03-17,Arkansas,5,22,0
4,2020-03-17,California,6,732,14
...,...,...,...,...,...
95,2020-03-18,Rhode Island,44,33,0
96,2020-03-18,South Carolina,45,60,1
97,2020-03-18,South Dakota,46,11,1
98,2020-03-18,Tennessee,47,98,0


In [8]:
df3 = df2[df2['state'] == 'West Virginia'].reset_index(drop=True)
print(len(df3))
df3.head(100)

624


,date,state,fips,cases,deaths
0,2020-03-17,West Virginia,54,1,0
1,2020-03-18,West Virginia,54,2,0
2,2020-03-19,West Virginia,54,5,0
3,2020-03-20,West Virginia,54,8,0
4,2020-03-21,West Virginia,54,12,0
...,...,...,...,...,...
95,2020-06-20,West Virginia,54,2500,88
96,2020-06-21,West Virginia,54,2543,89
97,2020-06-22,West Virginia,54,2571,90
98,2020-06-23,West Virginia,54,2593,92


In [9]:
df3['cases'][1]

2

In [10]:
statelist = sorted(list(set(df2['state']).intersection(set(states))))

print(len(statelist))
print(statelist)



51
['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']


In [11]:
datelist = sorted(list(set(df2['date'])))

print(len(datelist))

print(datelist)


624
['2020-03-17', '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-21', '2020-03-22', '2020-03-23', '2020-03-24', '2020-03-25', '2020-03-26', '2020-03-27', '2020-03-28', '2020-03-29', '2020-03-30', '2020-03-31', '2020-04-01', '2020-04-02', '2020-04-03', '2020-04-04', '2020-04-05', '2020-04-06', '2020-04-07', '2020-04-08', '2020-04-09', '2020-04-10', '2020-04-11', '2020-04-12', '2020-04-13', '2020-04-14', '2020-04-15', '2020-04-16', '2020-04-17', '2020-04-18', '2020-04-19', '2020-04-20', '2020-04-21', '2020-04-22', '2020-04-23', '2020-04-24', '2020-04-25', '2020-04-26', '2020-04-27', '2020-04-28', '2020-04-29', '2020-04-30', '2020-05-01', '2020-05-02', '2020-05-03', '2020-05-04', '2020-05-05', '2020-05-06', '2020-05-07', '2020-05-08', '2020-05-09', '2020-05-10', '2020-05-11', '2020-05-12', '2020-05-13', '2020-05-14', '2020-05-15', '2020-05-16', '2020-05-17', '2020-05-18', '2020-05-19', '2020-05-20', '2020-05-21', '2020-05-22', '2020-05-23', '2020-05-24', '2020-05-25', '2020-05-26', '

In [12]:

for i in range(len(statelist)):
    
    start_time = time.time()
    
    state = statelist[i]
        
    state_cases = []  # record cases for each state
    
    for j in range(len(datelist)):       
        
        df3 = df2[df2['state'] == state].reset_index(drop=True)        
        
        # state_cases.append(df3['cases'][j])
        state_cases.append(df3['deaths'][j])  
    
    
    state_cases = np.diff(state_cases, n=1)  # to calculate daily infected cases from cumulative cases   
    
    
    end_time = time.time()
    
    print("Time elapsed per state: ", end_time - start_time)
    
    with open(output_confirmed_inter, "a") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(state_cases)
        csv_file.close()
        
    print(state, 'is done')   
    

Time elapsed per state:  1.1377463340759277
Alabama is done
Time elapsed per state:  1.1761374473571777
Alaska is done
Time elapsed per state:  1.1478440761566162
Arizona is done
Time elapsed per state:  1.1230401992797852
Arkansas is done
Time elapsed per state:  1.090144157409668
California is done
Time elapsed per state:  1.126671314239502
Colorado is done
Time elapsed per state:  1.087658166885376
Connecticut is done
Time elapsed per state:  1.1273376941680908
Delaware is done
Time elapsed per state:  1.0765504837036133
District of Columbia is done
Time elapsed per state:  1.1048939228057861
Florida is done
Time elapsed per state:  1.1095402240753174
Georgia is done
Time elapsed per state:  1.0933022499084473
Hawaii is done
Time elapsed per state:  1.0857963562011719
Idaho is done
Time elapsed per state:  1.120976209640503
Illinois is done
Time elapsed per state:  1.109269142150879
Indiana is done
Time elapsed per state:  1.087087631225586
Iowa is done
Time elapsed per state:  1.09

In [13]:
df4 = pd.read_csv(output_confirmed_inter, header=None)
df4.head(100)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622
0,0,0,0,0,0,0,0,1,2,1,0,1,6,3,14,4,6,6,1,8,11,3,11,2,13,0,11,10,9,14,14,-4,13,4,22,15,-4,12,4,6,9,14,20,10,17,-1,2,8,17,28,26,14,7,3,10,32,15,23,10,2,3,1,15,18,7,12,8,2,15,14,3,8,19,8,12,16,7,0,0,23,13,3,26,11,15,11,14,4,0,1,11,5,20,12,16,1,2,23,27,5,11,12,0,10,21,22,13,21,1,0,0,26,25,10,36,10,7,3,40,47,19,35,21,1,4,12,61,33,41,18,17,18,0,47,27,15,23,24,6,33,29,19,21,20,13,29,50,35,8,3,3,2,27,11,8,30,22,15,2,11,13,8,31,31,45,10,20,18,17,16,33,9,1,0,1,8,16,32,17,1,4,32,5,9,27,9,0,2,18,31,18,-15,10,1,-1,16,23,8,2,8,0,1,21,21,36,16,11,0,1,0,41,50,30,2,0,1,16,23,15,16,7,0,0,26,19,3,18,35,6,0,14,19,20,23,33,2,0,36,81,12,18,15,2,1,52,46,72,32,6,0,2,13,60,40,0,0,5,1,60,73,65,55,46,12,3,50,43,49,52,16,0,0,22,74,56,42,93,0,0,63,135,89,4,5,6,21,25,37,53,45,0,1,5,8,108,86,111,108,35,13,226,187,185,85,89,1,1,5,157,96,107,171,3,2,234,276,168,226,0,122,0,206,309,162,84,64,2,8,56,309,133,159,62,0,2,13,89,78,149,17,2,0,68,84,87,38,61,-1,2,60,38,65,28,27,-1,0,38,36,52,25,28,0,2,8,26,28,34,11,0,0,14,37,17,19,3,0,0,1,27,-1,54,30,0,1,14,0,23,11,26,0,0,10,6,8,3,51,0,0,8,9,17,16,11,0,1,2,25,8,9,17,0,0,0,17,16,20,12,0,0,7,12,4,23,14,0,0,5,2,32,3,32,0,0,12,14,2,6,0,0,0,10,11,21,10,0,0,19,3,29,3,14,0,0,8,7,7,0,18,0,0,5,0,17,0,8,0,0,2,0,14,0,6,0,0,0,0,29,0,11,4,0,0,0,26,11,3,1,0,0,17,2,6,4,11,0,0,9,18,6,0,20,0,0,6,19,13,26,24,0,1,23,41,35,41,0,33,1,33,40,42,28,58,0,0,0,0,103,50,66,3,0,61,8,39,64,20,1,1,4,68,64,53,68,3,2,40,66,72,192,161,1,0,250,205,133,123,101,0,3,130,45,99,94,76,1,1,71,70,63,81,101,1,1,10,111,121,41,37,0,0,84,48,9,58,28,1,0,17,8,52,48,41,0,0,56,47,58,52,48,0,0,45,38,17,0,47,0,0,18,26,18,20,10,0,0,1

In [14]:
data = df4.iloc[:, :].values
data

array([[ 0,  0,  0, ...,  0,  1,  3],
       [ 0,  0,  0, ...,  0,  1,  2],
       [ 0,  0,  1, ...,  5,  1, 77],
       ...,
       [ 0,  0,  0, ...,  0, 20, 31],
       [ 0,  2,  1, ...,  0, 11, 51],
       [ 0,  0,  0, ...,  0,  0, 81]])

In [15]:
data = list(map(list, zip(*data)))
data

[[0,
  0,
  0,
  0,
  3,
  0,
  1,
  0,
  0,
  1,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  2,
  0,
  14,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  14,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  2,
  2,
  3,
  0,
  0,
  1,
  7,
  0,
  0,
  3,
  0,
  0,
  0,
  1,
  3,
  0,
  0,
  0,
  2,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  4,
  0,
  7,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  0,
  2,
  0,
  7,
  0,
  2,
  0],
 [0,
  0,
  1,
  0,
  5,
  0,
  0,
  0,
  0,
  1,
  4,
  0,
  0,
  1,
  3,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  2,
  0,
  0,
  1,
  0,
  2,
  0,
  29,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  2,
  0,
  0,
  1,
  0,
  0,
  0,
  8,
  0,
  1,
  0],
 [0,
  0,
  0,
  0,
  4,
  1,
  1,
  0,
  1,
  2,
  6,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  6,
  0,
  1,
  1,
  2,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  0,
  27,
  0,
  0,
  2,
  0,
  1,
  1,
  0,
  0,
  0,
  2,
  0,
  0,
  0

In [16]:
data = pd.DataFrame(data)

In [17]:
## X Matrix saved as a csv file
data.to_csv(output_confirmed_cases_US, header=0, index=0)

In [18]:
## X Matrix Generated

In [19]:
df5 = pd.read_csv(output_confirmed_cases_US, header=None)
df5.head(100)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50
0,0,0,0,0,3,0,1,0,0,1,2,0,0,0,0,0,0,0,2,0,1,0,1,0,0,1,0,0,0,0,2,0,14,0,0,0,0,1,1,0,0,0,0,1,0,0,0,14,0,0,0
1,0,0,0,0,2,2,3,0,0,1,7,0,0,3,0,0,0,1,3,0,0,0,2,0,1,0,0,0,0,0,4,0,7,0,0,0,1,0,0,0,0,0,0,2,0,2,0,7,0,2,0
2,0,0,1,0,5,0,0,0,0,1,4,0,0,1,3,0,0,0,0,0,1,1,1,0,0,2,0,0,1,0,2,0,29,0,0,1,0,0,0,0,2,0,0,1,0,0,0,8,0,1,0
3,0,0,0,0,4,1,1,0,1,2,6,0,0,1,1,0,1,1,6,0,1,1,2,1,0,0,0,0,0,0,5,0,27,0,0,2,0,1,1,0,0,0,2,0,0,0,1,12,0,1,0
4,0,0,1,0,7,1,3,0,0,2,5,0,0,3,3,0,0,0,4,0,0,3,3,0,0,0,0,0,0,0,4,0,47,0,0,0,1,1,1,0,0,0,0,2,1,0,3,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,0,3,1,22,0,9,1,2,17,1,0,0,19,4,1,1,2,1,0,14,30,3,8,0,0,0,3,1,0,13,3,45,6,1,3,1,2,5,0,9,0,3,9,3,0,4,3,1,0,0
96,2,0,4,2,44,4,3,0,2,12,4,0,0,24,13,1,2,5,12,0,8,16,7,4,35,5,1,3,3,0,25,0,50,5,0,4,0,2,3,9,6,0,6,18,0,0,9,6,1,0,0
97,23,0,44,10,76,14,14,69,2,65,39,0,0,36,16,3,1,10,17,0,18,16,12,9,11,13,0,7,2,4,54,7,36,26,1,31,2,0,36,3,14,2,10,33,5,0,25,8,2,7,0
98,27,0,69,3,91,2,10,1,4,43,12,0,1,61,9,4,1,2,18,1,16,48,5,7,22,8,0,1,2,4,46,4,31,19,2,20,1,3,57,6,10,1,13,46,0,0,16,9,0,11,0


In [20]:
os.remove(output_confirmed_inter)

# We used the same Adjacency Matrix for US states as generated from JHU Dataset